In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from keras.datasets import mnist
import seaborn as sns
import pandas as pd
import sys
from tqdm import tqdm
sys.path.append('../Deep_Learning/')
from DeepNeuralNetwork import DNN
from nn_optimization_methods import SGD
from layers.Dense import Dense
from layers.Input import Input
from layers.Softmax import Softmax
sys.path.append('../tools/')
import tools
import copy

%matplotlib inline

/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# AutoEncoder Example using DNN

## Demonstrated using mnist data

In [2]:
(X_train_digits, y_train), (X_test_digits, y_test) = mnist.load_data()

In [3]:
X_train = np.array(list(map(lambda x: x.flatten()/255,X_train_digits)))
X_test = np.array(list(map(lambda x: x.flatten()/255,X_test_digits)))

In [4]:
class AutoEncoder():
    """
    An auto encoder is a semi supervised learning algorithm that attempts to reconstruct input using a smaller feature space
    Parameters:
        X: numpy array(): data matrix
        encoder: DNN to reduce dimensions of matrix
        decoder: DNN to recreate the original data from the encoded data
        full_model: DNN that combines both the encoder and decoder objects, used to train both
    """
    def __init__(self,X):
        
        self.X = X
        self.encoder = None
        self.decoder = None
        self.full_model = DNN()
        self.full_model.add(Input(X))
        self.count = 0
        
    def create_encoder(self,layers=[Dense(16)],encoded_dims=2):
        
        self.count = 0
        for layer in layers:
            self.full_model.add(layer)
            self.count += 1
            
        self.full_model.add(Dense(encoded_dims))
        
    def create_decoder(self,layers=[]):
        
        if len(layers) > 0:
            for layer in layers:
                self.full_model.add(layer)
            
        self.full_model.add(Dense(self.X.shape[-1]))
    
    def finalize_encoder_decoder(self):
        
        count = 0
        layer = self.full_model.head.getNext()
        self.encoder = DNN()
        self.decoder = DNN()
        self.encoder.add(Input(self.X))
        
        while layer != None:
            print(layer)
            if count <= self.count:
                self.encoder.add(copy.deepcopy(layer))
                if count == a.count:
                    self.encoder.outlayer.next = None
                    self.decoder.add(Input(self.encoder.outlayer.output))
            else:
                self.decoder.add(layer)
            layer = layer.getNext()
            count += 1
            
    def train(self,learning_rate=0.001,epochs=100):
        
        self.full_model.train(self.X,self.X,lr=learning_rate,epochs=epochs)
        self.finalize_encoder_decoder()
    
    def predict(self,X):
        
        encoded = self.encoder.predict(X)
        decoded = self.decoder.predict(encoded)
        return encoded,decoded


In [5]:
a = AutoEncoder(X_train[:250])
a.create_encoder()
a.create_decoder()

In [6]:
print(a.full_model)

----------------------------- Model -----------------------------
Input: shape: (None, 784)
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (784, 16), output shape: (None, 16), parameters: 12544
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (16, 2), output shape: (None, 2), parameters: 32
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (2, 784), output shape: (None, 784), parameters: 1568
-----------------------------------------------------------------



In [7]:
a.train(epochs=1)

MSE 190.0758798537329: 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]

Dense: activation: sigmoid, weight shape: (784, 16), output shape: (None, 16), parameters: 12544
-----------------------------------------------------------------

Dense: activation: sigmoid, weight shape: (16, 2), output shape: (None, 2), parameters: 32
-----------------------------------------------------------------

Dense: activation: sigmoid, weight shape: (2, 784), output shape: (None, 784), parameters: 1568
-----------------------------------------------------------------



In [8]:
print(a.encoder)

----------------------------- Model -----------------------------
Input: shape: (None, 784)
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (784, 16), output shape: (None, 16), parameters: 12544
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (16, 2), output shape: (None, 2), parameters: 32
-----------------------------------------------------------------



In [9]:
print(a.decoder)

----------------------------- Model -----------------------------
Input: shape: (None, 2)
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (2, 784), output shape: (None, 784), parameters: 1568
-----------------------------------------------------------------

